In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "../../scripts/methods/")
from my_paste import paste_align, paste_align_ref
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

2024-07-17 05:06:49.780449: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 05:06:49.923933: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-17 05:06:49.923960: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-17 05:06:50.611677: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot ope

In [2]:
data_folder = "../../data/STARMapPlus/"
method = 'PASTE'
i = 3
adata = ad.read_h5ad(os.path.join(data_folder, f"sagittal{i}.h5ad"))

In [3]:
scale = np.linalg.norm(adata.obsm['spatial'].max(0) - adata.obsm['spatial'].min(0))

In [4]:
slices = split_slice(
    adata=adata,
    spatial_key='spatial_raw',
    split_num=2,
    axis=2,
)

In [5]:
overlay_ratio_set = np.arange(0.5, 1.05, 0.05)
from tqdm import tqdm
maes = []
for overlay_ratio in tqdm(overlay_ratio_set):
    slice1 = slices[0].copy()
    slice2 = slices[1].copy()
    slice1_crop, slice2_crop = crop_slices(
        slice1, 
        slice2, 
        spatial_key='spatial', 
        overlay_ratio = overlay_ratio,
    )
    slice1, slice2 = slice1_crop.copy(), slice2_crop.copy()
    sampling_num = 10000
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    spatial_key = 'spatial'
    key_added = 'align_spatial'
    
    align_slices, pis = paste_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        device="2",
        verbose=False,
    )
    spatial_align = align_slices[1].obsm[key_added]
    spatial_GT = align_slices[1].obsm['spatial']
    mae, mae_var = MAE(spatial_align, spatial_GT) / scale
    maes.append(mae)
np.save(f'./results/{method}_different_overlap.npy', {'maes': maes, 'overlay_ratio':overlay_ratio}, allow_pickle=True)

  0%|                                                                                                                                                                                                                                 | 0/11 [00:00<?, ?it/s]/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


gpu is available, using gpu.


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
  9%|███████████████████▋                                                                                                                                                                                                    | 1/11 [02:59<29:55, 179.59s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


gpu is available, using gpu.


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
 18%|███████████████████████████████████████▎                                                                                                                                                                                | 2/11 [06:53<31:41, 211.29s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


...

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
 82%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                       | 9/11 [27:29<05:14, 157.04s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


gpu is available, using gpu.


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
 91%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                   | 10/11 [29:59<02:34, 154.76s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/ParametersStudies/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


gpu is available, using gpu.


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [31:30<00:00, 171.90s/it]
